In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import skew, kurtosis
import glob
import os

In [3]:
custinfo = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_custinfo_full_hashed.csv')
cdtx = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_cdtx0001_full_hashed.csv')
dp = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_dp_full_hashed.csv')
remit = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_train_x_remit1_full_hashed.csv')
alert_pub = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/public_x_alert_date.csv')
alert_train = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/train_x_alert_date.csv')
alert =pd.concat([alert_pub, alert_train])
label = pd.read_csv('E:/Datasets/Fintech/TrainingDataset_first/train_y_answer.csv')

In [4]:
custs = custinfo.cust_id.unique()
rng = np.arange(0,395)
win_len = 5
step = 1

In [5]:
def calculate_period_feats(txs, diff_thres=1000):
    diff1 = np.diff(txs)
    norm_diff1 = np.diff((txs-np.mean(txs))/(np.std(txs)+1e-20))

    meanTX = np.mean(txs)
    stdTX = np.std(txs)
    cvTX = stdTX/(meanTX+1e-20)
    diff1000 = 0 if len(diff1)==0 else sum(np.abs(diff1)>diff_thres)/len(diff1)
    mean_diff1 = 0 if len(diff1)==0 else np.mean(diff1)
    std_abs_diff1 = 0 if len(diff1)==0 else np.std(np.abs(diff1))
    rmsdd = 0 if len(diff1)==0 else np.sqrt(np.mean(np.power(diff1, 2)))
    mean_abs_diff1 = 0 if len(diff1)==0 else np.mean(np.abs(norm_diff1))
    nmae1_diff1 = 0 if len(diff1)==0 else np.mean(np.abs(diff1))/(meanTX+1e-20) # normalized mean of absolute 1st diff
    feats = [
            meanTX,
            stdTX,
            cvTX,
            diff1000,
            mean_diff1,
            std_abs_diff1,
            rmsdd,
            mean_abs_diff1,
            nmae1_diff1,
                ]
    return feats

def aggregate_feats(period, ak):
    period_features = []
    period_mean = period.apply(lambda x: np.mean(x)).values
    period_std = period.apply(lambda x: np.std(x)).values
    period_cv = period_std/(period_mean+1e-20)
    period_median = period.apply(lambda x: np.median(x)).values
    period_min = period.apply(lambda x: np.min(x)).values
    period_max = period.apply(lambda x: np.max(x)).values
    period_1quartile = period.apply(lambda x: np.percentile(x, 25)).values
    period_3quartile = period.apply(lambda x: np.percentile(x, 75)).values
    period_skew = period.apply(lambda x: skew(x)).values
    period_kurt = period.apply(lambda x: kurtosis(x)).values

    period_weighted = period.apply(lambda x: x.iloc[:-1]*x.iloc[-1], axis=1)
    period_mean_weighted = period_weighted.apply(lambda x: np.mean(x)).values
    period_std_weighted = period_weighted.apply(lambda x: np.std(x)).values
    period_cv_weighted = period_std_weighted/(period_mean_weighted+1e-20)

    # print(period_mean.shape)
    # print(period_std.shape)
    # print(period_cv.shape)
    # print(period_median.shape)
    # print(period_min.shape)
    # print(period_max.shape)
    # print(period_1quartile.shape)
    # print(period_3quartile.shape)
    # print(period_skew.shape)
    # print(period_kurt.shape)
    # print(period_mean_weighted.shape)
    # print(period_std_weighted.shape)
    # print(period_cv_weighted.shape)
    period_features = np.hstack((
        period_mean,
        period_std,
        period_cv,
        period_median,
        period_min,
        period_max,
        period_1quartile,
        period_3quartile,
        period_skew,
        period_kurt,
        period_mean_weighted,
        period_std_weighted,
        period_cv_weighted,
        ak
    )).reshape(1,-1)

    return period_features


## CDTX

In [6]:
for cust in tqdm(custs):
    aks_c = alert[alert.alert_key.isin(custinfo[custinfo.cust_id==cust].alert_key)].sort_values(by=['date'])
    cdtx_c = cdtx[cdtx.cust_id==cust].sort_values(by=['date'])
    features = []
    ts = []
    idx = win_len-1
    if len(cdtx_c) > 0:
        while idx <= rng[-1]:
            win_data = cdtx_c[(cdtx_c.date>=(idx-win_len+1)) & (cdtx_c.date<=(idx))]
            if len(win_data) > 0:
                txs = win_data['amt'].values
                feats = calculate_period_feats(txs)
                feats.append(len(txs))

                features.append(feats)
                ts.append(idx)
            idx += step

        df_TR = pd.DataFrame(features)
        df_TR['date'] = ts

        # Aggregate from the beginning to alert key date
        period_features = []
        for ak in aks_c.alert_key:
            ak_d = aks_c[aks_c.alert_key==ak].date.item()
            period = df_TR[df_TR.date<=ak_d].iloc[:,:-1]
            if len(period) > 0:
                period_feats = aggregate_feats(period, ak)
                assert not np.isnan(period_feats).any(), "Should not have NaN"
                period_features.append(period_feats)
        
        if len(period_features) > 0: # there may be no tx data before a alert key issued
            period_features = np.vstack(period_features)
            output = pd.DataFrame(period_features)
            output.rename(columns={output.columns[-1]:'alert_key'}, inplace=True)

            outname = f"E:/Datasets/Fintech/AggregatedData/cdtx/{cust}.csv"
            output.to_csv(outname, index=False)
    

100%|██████████| 7708/7708 [41:27<00:00,  3.10it/s]  


## DP

why on earth are there NaNs in tx_amt???? fk u

In [7]:
dp['tx_amt'].fillna(0, inplace=True)
for cust in tqdm(custs):
    aks_c = alert[alert.alert_key.isin(custinfo[custinfo.cust_id==cust].alert_key)].sort_values(by=['date'])
    dp_c = dp[dp.cust_id==cust].sort_values(by=['tx_date'])
    dp_c.rename(columns={'tx_date': 'date'}, inplace=True)
    features = []
    ts = []
    idx = win_len-1
    if len(dp_c) > 0:
        while idx <= rng[-1]:
            win_data = dp_c[(dp_c.date>=(idx-win_len+1)) & (dp_c.date<=(idx))]
            if len(win_data) > 0:
                txs = win_data['tx_amt'].values * win_data['exchg_rate'].values
                feats = calculate_period_feats(txs)
                feats.append(sum(win_data.cross_bank==1)/len(txs)) # cross bank ratio
                feats.append(len(txs))

                features.append(feats)
                ts.append(idx)
            idx += step

        df_TR = pd.DataFrame(features)
        df_TR['date'] = ts

        # Aggregate from the beginning to alert key date
        period_features = []
        for ak in aks_c.alert_key:
            ak_d = aks_c[aks_c.alert_key==ak].date.item()
            period = df_TR[df_TR.date<=ak_d].iloc[:,:-1]
            if len(period) > 0:
                period_feats = aggregate_feats(period, ak)
                assert not np.isnan(period_feats).any(), "Should not have NaN"
                period_features.append(period_feats)
        
        if len(period_features) > 0: # there may be no tx data before a alert key issued
            period_features = np.vstack(period_features)
            output = pd.DataFrame(period_features)
            output.rename(columns={output.columns[-1]:'alert_key'}, inplace=True)

            outname = f"E:/Datasets/Fintech/AggregatedData/dp/{cust}.csv"
            output.to_csv(outname, index=False)
    
    

100%|██████████| 7708/7708 [1:07:18<00:00,  1.91it/s]


## Remit

In [8]:
for cust in tqdm(custs):
    aks_c = alert[alert.alert_key.isin(custinfo[custinfo.cust_id==cust].alert_key)].sort_values(by=['date'])
    remit_c = remit[remit.cust_id==cust].sort_values(by=['trans_date'])
    remit_c.rename(columns={'trans_date': 'date'}, inplace=True)
    features = []
    ts = []
    idx = win_len-1
    if len(remit_c) > 0:
        while idx <= rng[-1]:
            win_data = remit_c[(remit_c.date>=(idx-win_len+1)) & (remit_c.date<=(idx))]
            if len(win_data) > 0:
                txs = win_data['trade_amount_usd'].values
                feats = calculate_period_feats(txs, diff_thres=40)
                feats.append(len(txs))

                features.append(feats)
                ts.append(idx)
            idx += step

        df_TR = pd.DataFrame(features)
        df_TR['date'] = ts

        # Aggregate from the beginning to alert key date
        period_features = []
        for ak in aks_c.alert_key:
            ak_d = aks_c[aks_c.alert_key==ak].date.item()
            period = df_TR[df_TR.date<=ak_d].iloc[:,:-1]
            if len(period) > 0:
                period_feats = aggregate_feats(period, ak)
                assert not np.isnan(period_feats).any(), "Should not have NaN"
                period_features.append(period_feats)
        
        if len(period_features) > 0: # there may be no tx data before a alert key issued
            period_features = np.vstack(period_features)
            output = pd.DataFrame(period_features)
            output.rename(columns={output.columns[-1]:'alert_key'}, inplace=True)

            outname = f"E:/Datasets/Fintech/AggregatedData/remit/{cust}.csv"
            output.to_csv(outname, index=False)
    

100%|██████████| 7708/7708 [10:06<00:00, 12.71it/s] 


## Merge 3 types of features

In [11]:
all_data = []
for cust in tqdm(custs):
    cdtx_c = None
    dp_c = None
    remit_c = None
    aks_c = []
    if os.path.exists(f"E:/Datasets/Fintech/AggregatedData/cdtx/{cust}.csv"):
        cdtx_c = pd.read_csv(f"E:/Datasets/Fintech/AggregatedData/cdtx/{cust}.csv")
        aks_c.append(cdtx_c.alert_key.unique())
    if os.path.exists(f"E:/Datasets/Fintech/AggregatedData/dp/{cust}.csv"):
        dp_c = pd.read_csv(f"E:/Datasets/Fintech/AggregatedData/dp/{cust}.csv")
        aks_c.append(dp_c.alert_key.unique())
    if os.path.exists(f"E:/Datasets/Fintech/AggregatedData/remit/{cust}.csv"):
        remit_c = pd.read_csv(f"E:/Datasets/Fintech/AggregatedData/remit/{cust}.csv")
        aks_c.append(remit_c.alert_key.unique())
    if len(aks_c) > 0:
        aks_c = np.unique(np.hstack((aks_c)))
        for ak_c in aks_c:
            try:
                cdtx_data = cdtx_c[cdtx_c.alert_key==ak_c].values[:,:-1].flatten()
                if len(cdtx_data)==0:
                    cdtx_data = np.zeros(127)
            except:
                cdtx_data = np.zeros(127)
            try:
                dp_data = dp_c[dp_c.alert_key==ak_c].values[:,:-1].flatten()
                if len(dp_data)==0:
                    dp_data = np.zeros(140)
            except:
                dp_data = np.zeros(140)
            try:
                remit_data = remit_c[remit_c.alert_key==ak_c].values[:,:-1].flatten()
                if len(remit_data)==0:
                    remit_data = np.zeros(127)    
            except:
                remit_data = np.zeros(127)
            all_data.append(np.hstack((cdtx_data, dp_data, remit_data, ak_c)).reshape(1,-1))

all_data = np.vstack(all_data)
ds_out = pd.DataFrame(all_data)
ds_out.rename(columns={ds_out.columns[-1]:'alert_key'}, inplace=True)

outname = f"E:/Datasets/Fintech/AggregatedData/timeSeriesStats.csv"
ds_out.to_csv(outname, index=False)

    
        

100%|██████████| 7708/7708 [01:47<00:00, 71.99it/s] 


In [10]:
ds_out

,0,1,2,3,4,5,6,7,8,9,...,385,386,387,388,389,390,391,392,393,alert_key
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,172802.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,174202.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,174952.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177764.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178768.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23730,1449.137166,451.837772,0.362254,0.093134,294.389884,270.200949,773.098536,0.916063,0.463766,2.864629,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351904.0
23731,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351926.0
23732,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351929.0
23733,1295.195270,1026.864463,0.558325,0.243689,-9.991591,702.163035,1655.507187,1.265892,0.747936,3.690751,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,352103.0


In [207]:
ds_out.columns[ds_out.isna().any(axis=0)]

Index([], dtype='object')

## Complete

In [22]:
label[label.sar_flag==1]

,alert_key,sar_flag
44,171770,1
136,172480,1
448,174298,1
1609,177960,1
1611,177788,1
...,...,...
23164,346021,1
23308,347051,1
23368,347894,1
23572,348724,1


In [36]:
alert[alert.alert_key.isin(label[label.sar_flag==1].alert_key)]

,alert_key,date
44,171770,0
136,172480,5
448,174298,7
1609,177960,12
1611,177788,12
...,...,...
23164,346021,351
23308,347051,355
23368,347894,356
23572,348724,358


In [29]:
custinfo[custinfo.alert_key==172480].cust_id.values

array(['7fde997da7e7a95280460889004bd2f23050bd8953de7ea648bd3fd9fc977fde'],
      dtype=object)

In [30]:
custinfo[custinfo.cust_id=='7fde997da7e7a95280460889004bd2f23050bd8953de7ea648bd3fd9fc977fde']

,alert_key,cust_id,risk_rank,occupation_code,total_asset,AGE
2026,172480,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,3,12.0,8251.0,4


In [32]:
dp[dp.cust_id=='7fde997da7e7a95280460889004bd2f23050bd8953de7ea648bd3fd9fc977fde'].sort_values(by=['tx_date'])

,cust_id,debit_credit,tx_date,tx_time,tx_type,tx_amt,exchg_rate,info_asset_code,fiscTxId,txbranch,cross_bank,ATM
1216120,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,DB,0,15,2,22164.0,1.0,13,4.0,0.0,0,1
1216087,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,DB,0,15,2,51523.0,1.0,13,4.0,0.0,0,1
1216117,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,CR,0,15,1,2575.0,1.0,13,20.0,133.0,0,1
1216115,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,DB,0,15,2,21958.0,1.0,13,4.0,0.0,0,1
1216112,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,CR,0,15,1,7432.0,1.0,13,20.0,133.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1216127,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,DB,375,1,2,11950.0,1.0,2,NaN,NaN,0,1
1216129,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,CR,375,1,2,1339.0,1.0,13,4.0,169.0,1,1
1216123,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,CR,387,13,2,2388.0,1.0,13,4.0,279.0,1,1
1216124,7fde997da7e7a95280460889004bd2f23050bd8953de7e...,DB,390,9,1,412.0,1.0,13,21.0,133.0,0,1
